In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '/kaggle/input/applied-ml-microcourse-telco-churn'
data = pd.read_csv('{}/features.csv'.format(path))

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

y = data['churn']

X = data.drop(columns=['customerID', 'churn'])
X = pd.get_dummies(X, drop_first=True)

# some variables are highly/exactly correlated.  Dropping additional variables
vars_to_drop = ['MultipleLines_No phone service',
                'OnlineSecurity_No internet service',
                'OnlineBackup_No internet service',
                'DeviceProtection_No internet service',
                'TechSupport_No internet service',
                'StreamingTV_No internet service',
                'StreamingMovies_No internet service']

X.drop(columns=vars_to_drop, inplace=True)
X.fillna(0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn import metrics

def get_performance_measures(clf, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    test_pred = clf.predict(X_test)
    train_pred = clf.predict(X_train)
    
    print("\nClassification report (test):\n", metrics.classification_report(y_test, test_pred))
    print("\nROC AUC (test):\n", round(metrics.roc_auc_score(y_test, test_pred), 2), "\n")

    print("\nClassification report (train):\n", metrics.classification_report(y_train, train_pred))
    print("\nROC AUC (train):\n", round(metrics.roc_auc_score(y_train, train_pred), 2), "\n")

### Logistic Regression Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
clf_lr.fit(X_train, y_train)

get_performance_measures(clf_lr)

Read through the error above. It is often advisable to scale features for models like logistic regression.

In [ ]:
from sklearn import preprocessing

X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

clf_lr = LogisticRegression()
clf_lr.fit(X_train_scaled, y_train)

get_performance_measures(clf_lr, X_train=X_train_scaled, X_test=X_test_scaled)

The performance is similar after we scale the data, but we know the model converged at least.

### Random Forest Modelling

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)

get_performance_measures(clf_rf)

We see that the test ROC AUC for the random forest is 72.7%, lower than the 73.4% from the logistic regression.  However, the training ROC AUC from the random forest is 100%.  This is clear evidence that overfitting has occured.  We can address this by changing some hyperparameters

In [ ]:
clf_rf = RandomForestClassifier(min_samples_leaf=10)
clf_rf.fit(X_train, y_train)

get_performance_measures(clf_rf)

### Try XGBoost, widely regarded as a workhorse of modern machine learning

In [ ]:
from xgboost import XGBClassifier

clf_xgb = XGBClassifier()
clf_xgb.fit(X_train, y_train)

get_performance_measures(clf_xgb)

We're getting overfitting with the default parameters for XGBoost, but our test ROC AUC is higher than both the logistic regression and random forest models.  Let's try making the model less complex by altering a hyperparameter.

In [ ]:
clf_xgb = XGBClassifier(min_child_weight=5, max_depth=2)
clf_xgb.fit(X_train, y_train)

get_performance_measures(clf_xgb)

We are able to get better performance with the XGBoost model, although there is still some overfitting.  There are more hyperparamaters we can choose, so we may be able to get a better model.